In [44]:
import os
import cv2
import numpy as np
import mediapipe as mp
import librosa
import soundfile as sf
from tqdm import tqdm
from moviepy.editor import VideoFileClip
import warnings
warnings.filterwarnings("ignore")

# Parameters
clip_len = 150
lip_landmark_ids = list(range(61, 81))  # 20 lip landmarks

# Initialize Mediapipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Input/output
input_root = "dataset"
output_root = "processed_dataset"
os.makedirs(output_root, exist_ok=True)

def extract_audio_from_video(video_path, temp_wav_path):
    try:
        clip = VideoFileClip(video_path)
        clip.audio.write_audiofile(temp_wav_path, verbose=False, logger=None)
        return True
    except Exception:
        return False

def extract_lip_landmarks_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    landmarks = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb)

        if results.multi_face_landmarks:
            face = results.multi_face_landmarks[0]
            points = [(lmk.x, lmk.y) for i, lmk in enumerate(face.landmark) if i in lip_landmark_ids]
            landmarks.append(points)

    cap.release()
    return np.array(landmarks)  # shape: (T, 20, 2)

def extract_mfcc(temp_wav_path, fps, total_frames):
    try:
        y, sr = librosa.load(temp_wav_path, sr=None)
        duration = total_frames / fps
        y = y[:int(sr * duration)]

        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfcc = mfcc.T  # shape (n_frames, 13)

        # Align MFCC to video FPS using linear interpolation
        indices = np.linspace(0, len(mfcc) - 1, total_frames).astype(int)
        mfcc_aligned = mfcc[indices]
        return mfcc_aligned  # shape: (total_frames, 13)
    except Exception:
        return None

# Process each class
for cls in os.listdir(input_root):
    class_path = os.path.join(input_root, cls)
    if not os.path.isdir(class_path): continue

    out_frame_dir = os.path.join(output_root, cls, "frames")
    out_audio_dir = os.path.join(output_root, cls, "audio_features")
    os.makedirs(out_frame_dir, exist_ok=True)
    os.makedirs(out_audio_dir, exist_ok=True)

    for filename in tqdm(os.listdir(class_path), desc=f"Processing {cls}"):
        if not filename.endswith(".mp4"):
            continue

        vid_path = os.path.join(class_path, filename)
        temp_wav_path = "temp.wav"
        name_base = os.path.splitext(filename)[0]

        # Extract landmarks
        frames = extract_lip_landmarks_from_video(vid_path)
        if len(frames) < clip_len:
            # print(f"[SKIP] {filename} has only {len(frames)} frames.")
            continue

        # Extract aligned audio
        success = extract_audio_from_video(vid_path, temp_wav_path)
        if not success:
            print(f"[ERROR] Audio extract error in {filename}")
            continue

        fps = cv2.VideoCapture(vid_path).get(cv2.CAP_PROP_FPS)
        mfcc_full = extract_mfcc(temp_wav_path, fps, len(frames))
        if mfcc_full is None or mfcc_full.shape[0] < clip_len:
            print(f"[ERROR] MFCC extraction error in {filename}")
            continue

        # Save first valid clip of 150 frames
        lip_clip = frames[:clip_len]
        mfcc_clip = mfcc_full[:clip_len]

        # Save
        np.save(os.path.join(out_frame_dir, f"{name_base}_clip0_frame.npy"), lip_clip)
        np.save(os.path.join(out_audio_dir, f"{name_base}_clip0_af.npy"), mfcc_clip)


Processing fake: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 4206/4206 [1:32:51<00:00,  1.32s/it]
Processing .ipynb_checkpoints: 0it [00:00, ?it/s]


In [51]:
class LipSyncDataset(Dataset):
    def __init__(self, root_dir):
        self.data = []
        self.labels = []
        self.root_dir = root_dir
        classes = {'real': 0, 'fake': 1}
        
        for cls in classes:
            frame_dir = os.path.join(root_dir, cls, 'frames')
            audio_dir = os.path.join(root_dir, cls, 'audio_features')
            
            for fname in os.listdir(frame_dir):
                if not fname.endswith("_clip0_frame.npy"):
                    continue
                base = fname.replace("_clip0_frame.npy", "")
                frame_path = os.path.join(frame_dir, fname)
                audio_path = os.path.join(audio_dir, f"{base}_clip0_af.npy")
                
                if os.path.exists(audio_path):
                    self.data.append((frame_path, audio_path))
                    self.labels.append(classes[cls])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        frame_path, audio_path = self.data[idx]
        frames = np.load(frame_path)              # (150, 20, 2)
        audio = np.load(audio_path)               # (150, 13)
        label = self.labels[idx]

        # To tensors
        frames = torch.tensor(frames, dtype=torch.float32)  # (150, 20, 2)
        audio = torch.tensor(audio, dtype=torch.float32)    # (150, 13)
        return frames, audio, torch.tensor(label)


In [63]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from tqdm import tqdm

# =====================
# Dataset Loader
# =====================
class LipSyncDataset(Dataset):
    def __init__(self, root_dir):
        self.data = []
        self.labels = []
        self.root_dir = root_dir
        classes = {'real': 0, 'fake': 1}
        
        for cls in classes:
            frame_dir = os.path.join(root_dir, cls, 'frames')
            audio_dir = os.path.join(root_dir, cls, 'audio_features')
            
            for fname in os.listdir(frame_dir):
                if not fname.endswith("_clip0_frame.npy"):
                    continue
                base = fname.replace("_clip0_frame.npy", "")
                frame_path = os.path.join(frame_dir, fname)
                audio_path = os.path.join(audio_dir, f"{base}_clip0_af.npy")
                
                if os.path.exists(audio_path):
                    self.data.append((frame_path, audio_path))
                    self.labels.append(classes[cls])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        frame_path, audio_path = self.data[idx]
        frames = np.load(frame_path)              # (150, 20, 2)
        audio = np.load(audio_path)               # (150, 13)
        label = self.labels[idx]

        frames = torch.tensor(frames, dtype=torch.float32)  # (150, 20, 2)
        audio = torch.tensor(audio, dtype=torch.float32)    # (150, 13)
        return frames, audio, torch.tensor(label)

# =====================
# Model Definition
# =====================
class LipSyncLSTMClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        # Lip branch
        self.lip_lstm = nn.LSTM(input_size=40, hidden_size=64, batch_first=True)

        # Audio branch
        self.audio_lstm = nn.LSTM(input_size=13, hidden_size=64, batch_first=True)

        # Classifier
        self.fc = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # Real or Fake
        )

    def forward(self, lips, audio):
        B = lips.shape[0]
        lips = lips.view(B, 150, -1)  # (B, 150, 40)

        _, (h_lip, _) = self.lip_lstm(lips)      # (1, B, 64)
        _, (h_audio, _) = self.audio_lstm(audio) # (1, B, 64)

        fused = torch.cat([h_lip[-1], h_audio[-1]], dim=1)  # (B, 128)
        return self.fc(fused)

# =====================
# Training Setup
# =====================
def train_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = LipSyncDataset("processed_dataset")
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_set, val_set = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=16)

    model = LipSyncLSTMClassifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    num_epochs = 50
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        train_correct = 0

        for lips, audio, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            lips, audio, labels = lips.to(device), audio.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(lips, audio)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_correct += (outputs.argmax(1) == labels).sum().item()

        train_acc = train_correct / len(train_set)
        print(f"[Train] Epoch {epoch+1}: Loss = {running_loss:.4f}, Accuracy = {train_acc:.4f}")

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for lips, audio, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                lips, audio, labels = lips.to(device), audio.to(device), labels.to(device)
                outputs = model(lips, audio)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += (outputs.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_set)
        print(f"[Val]   Epoch {epoch+1}: Loss = {val_loss:.4f}, Accuracy = {val_acc:.4f}")

    # Save model
    torch.save(model.state_dict(), "lipsync_deepfake_model.pth")
    print("Model saved as lipsync_deepfake_model.pth")

# Run training
if __name__ == "__main__":
    train_model()


Epoch 1/50 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:12<00:00, 14.66it/s]


[Train] Epoch 1: Loss = 130.6889, Accuracy = 0.5458


Epoch 1/50 - Validation: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:01<00:00, 39.45it/s]


[Val]   Epoch 1: Loss = 32.7551, Accuracy = 0.5639


Epoch 2/50 - Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:13<00:00, 13.98it/s]


...

[Val]   Epoch 49: Loss = 23.8594, Accuracy = 0.7628


Epoch 50/50 - Training: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:11<00:00, 16.90it/s]


[Train] Epoch 50: Loss = 69.2596, Accuracy = 0.8295


Epoch 50/50 - Validation: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:01<00:00, 37.67it/s]

[Val]   Epoch 50: Loss = 23.3876, Accuracy = 0.7694
Model saved as lipsync_deepfake_model.pth


In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import librosa
from moviepy.editor import VideoFileClip
import mediapipe as mp

# ------------------------------
# Define model again (should match training)
# ------------------------------
class LipSyncLSTMClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        # Lip branch
        self.lip_lstm = nn.LSTM(input_size=40, hidden_size=64, batch_first=True)

        # Audio branch
        self.audio_lstm = nn.LSTM(input_size=13, hidden_size=64, batch_first=True)

        # Classifier
        self.fc = nn.Sequential(
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)  # Real or Fake
        )

    def forward(self, lips, audio):
        B = lips.shape[0]
        lips = lips.view(B, 150, -1)  # (B, 150, 40)

        _, (h_lip, _) = self.lip_lstm(lips)      # (1, B, 64)
        _, (h_audio, _) = self.audio_lstm(audio) # (1, B, 64)

        fused = torch.cat([h_lip[-1], h_audio[-1]], dim=1)  # (B, 128)
        return self.fc(fused)

# ------------------------------
# Load trained model weights
# ------------------------------
model = LipSyncLSTMClassifier()
model.load_state_dict(torch.load("lipsync_deepfake_model.pth", map_location=torch.device('cpu')))
model.eval()

# ------------------------------
# MediaPipe for lip landmarks
# ------------------------------
mp_face_mesh = mp.solutions.face_mesh
lip_landmarks = list(range(61, 81))  # 20 lip landmarks

def extract_lip_landmarks(video_path, max_frames=150):
    cap = cv2.VideoCapture(video_path)
    lips = []

    with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1) as face_mesh:
        while len(lips) < max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb)
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    points = []
                    for idx in lip_landmarks:
                        lm = face_landmarks.landmark[idx]
                        points.extend([lm.x, lm.y])  # 2 values per landmark
                    lips.append(points)
                    break
        cap.release()

    lips = np.array(lips)
    if len(lips) < max_frames:
        pad = np.zeros((max_frames - len(lips), 40))
        lips = np.vstack([lips, pad])
    else:
        lips = lips[:max_frames]
    return lips  # shape [150, 40]

# ------------------------------
# Extract audio features (MFCC)
# ------------------------------
def extract_audio_from_video(video_path, temp_wav_path="temp.wav"):
    try:
        clip = VideoFileClip(video_path)
        clip.audio.write_audiofile(temp_wav_path, verbose=False, logger=None)
        return temp_wav_path
    except Exception as e:
        print(f"[ERROR] Audio extract failed: {e}")
        return None

def extract_audio_features(video_path, max_frames=150):
    temp_wav_path = extract_audio_from_video(video_path)
    if temp_wav_path is None:
        return np.zeros((max_frames, 13))  # Adjusted for n_mfcc=13

    y, sr = librosa.load(temp_wav_path, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # ✅ correct number of MFCCs
    mfcc = mfcc.T  # [T, 13]

    if len(mfcc) < max_frames:
        pad = np.zeros((max_frames - len(mfcc), 13))
        mfcc = np.vstack([mfcc, pad])
    else:
        mfcc = mfcc[:max_frames]
    return mfcc

# ------------------------------
# Inference function
# ------------------------------
def predict(video_path):
    lips = extract_lip_landmarks(video_path)  # [150, 40]
    audio = extract_audio_features(video_path)  # [150, 13]

    lips_tensor = torch.tensor(lips, dtype=torch.float32).unsqueeze(0)   # [1, 150, 40]
    audio_tensor = torch.tensor(audio, dtype=torch.float32).unsqueeze(0) # [1, 150, 13]

    with torch.no_grad():
        output = model(lips_tensor, audio_tensor)
        probs = torch.softmax(output, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        conf = probs[0, pred].item()
        print(f"Prediction: {'FAKE' if pred == 1 else 'REAL'} (Confidence: {conf:.2f})")

# ------------------------------
# Run on a test video
# ------------------------------
if __name__ == "__main__":
    test_video = "fake/fake2.mp4"  # Replace with your video path
    predict(test_video)


[ERROR] Audio extract failed: 'NoneType' object has no attribute 'write_audiofile'
Prediction: FAKE (Confidence: 1.00)
